In [48]:
#hybrid search using pinecone and bm25Encoder

In [ ]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
#this class is responsible for doing semantic and keyword search in given docs

In [28]:
#creating an index
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

In [29]:
api_keys=os.getenv("PINECONE_API_KEY")

In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
pc=Pinecone(api_key=api_keys)
pc

In [32]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),)


In [33]:
index=pc.Index(index_name)
index

In [34]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [35]:
from langchain_huggingface import HuggingFaceEmbeddings

In [36]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [37]:
from pinecone_text.sparse import BM25Encoder

In [38]:

bm25_encoder=BM25Encoder().default()

In [39]:
sentences=[
    "i am studying 12th in ariyalur",
    "i am studying my ug graduate in trichy",
    "i am working in banglore",
    "my family is poor",
    "i score 451 marks in 10th standard",
    "i score 480 marks in 11th standard",
    "i score 550 marks in 12 th standard"
]

In [40]:
#tfidf values on these sentences
bm25_encoder.fit(sentences)

100%|██████████| 7/7 [00:00<?, ?it/s]


In [41]:
import json

In [42]:
bm25_encoder.dump("my_personal.json")

In [43]:
bm25_encoder=BM25Encoder().load("my_personal.json")

In [44]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [45]:
#add the sentence inside my idex
retriever.add_texts([
    "i am studying 12th in ariyalur",
    "i am studying my ug graduate in trichy",
    "i am working in banglore",
    "my family is poor",
    "i score 451 marks in 10th standard",
    "i score 480 marks in 11th standard",
    "i score 550 marks in 12 th standard"
])

100%|██████████| 1/1 [00:02<00:00,  2.78s/it]


In [47]:
retriever.invoke("which city i was working")                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

[Document(metadata={'score': 0.325875521}, page_content='i am working in banglore'),
 Document(metadata={'score': 0.318535596}, page_content="i'm working in banglore"),
 Document(metadata={'score': 0.0862972438}, page_content="i'm studying my ug graduate in trichy"),
 Document(metadata={'score': 0.0746321678}, page_content='i score 451 marks in 10th standard')]